In [ ]:
from google.colab import files
import pandas as pd
import io


uploaded = files.upload()


df = pd.read_csv(io.BytesIO(uploaded['nano_ai_final_aug.csv']))
print(df.head())


Saving nano_ai_final_aug.csv to nano_ai_final_aug.csv
   Nano_Silica (%)       W/C  Plasticizer  Cement (kg/m³)  Fine_Agg (kg/m³)  \
0         3.170000  0.480000     0.710000      447.800000        527.100000   
1         1.452613  0.693848     0.286133      362.380948        638.629896   
2         2.688229  0.580481     1.013763      480.496924        767.253862   
3         4.192975  0.449628     0.175303      443.493451        688.191828   
4         1.242139  0.391894     0.536623      449.401746        725.742176   

   Coarse_Agg (kg/m³)  Strength (N/mm²)  
0         1035.500000         26.010000  
1         1169.643246         28.598149  
2         1122.439172         23.254848  
3         1142.478188         26.412861  
4         1397.710087         30.661000  


In [ ]:
# Check column names
print(df.columns)


Index(['Nano_Silica (%)', 'W/C', 'Plasticizer', 'Cement (kg/m³)',
       'Fine_Agg (kg/m³)', 'Coarse_Agg (kg/m³)', 'Strength (N/mm²)'],
      dtype='object')


In [ ]:
# Features
X = df.drop(columns=['Strength (N/mm²)']).values

# Target
y = df['Strength (N/mm²)'].values

print("Features shape:", X.shape)
print("Target shape:", y.shape)


Features shape: (952, 6)
Target shape: (952,)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for RNN/LSTM: (samples, timesteps, features)
X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

print("X_train_rnn shape:", X_train_rnn.shape)
print("X_test_rnn shape:", X_test_rnn.shape)


X_train_rnn shape: (761, 1, 6)
X_test_rnn shape: (191, 1, 6)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# ----------------------
# 1️⃣ Pure RNN
# ----------------------
rnn_model = Sequential([
    Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    SimpleRNN(64, activation='tanh'),
    Dense(32, activation='relu'),
    Dense(1)
])

rnn_model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])
rnn_model.fit(X_train_rnn, y_train, validation_data=(X_test_rnn, y_test),
              epochs=200, batch_size=32, callbacks=[early_stop], verbose=0)

y_pred_rnn = rnn_model.predict(X_test_rnn)
r2_rnn = r2_score(y_test, y_pred_rnn)
print("Pure RNN R² Score:", r2_rnn)


# ----------------------
# 2️⃣ LSTM RNN
# ----------------------
lstm_model = Sequential([
    Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    LSTM(64, activation='tanh'),
    Dense(32, activation='relu'),
    Dense(1)
])

lstm_model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])
lstm_model.fit(X_train_rnn, y_train, validation_data=(X_test_rnn, y_test),
               epochs=200, batch_size=32, callbacks=[early_stop], verbose=0)

y_pred_lstm = lstm_model.predict(X_test_rnn)
r2_lstm = r2_score(y_test, y_pred_lstm)
print("LSTM RNN R² Score:", r2_lstm)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Pure RNN R² Score: 0.45605500427216916


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
LSTM RNN R² Score: 0.545050899569756


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn  = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [ ]:
import tensorflow as tf
import numpy as np
import random

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Pure RNN example
rnn_model = Sequential([
    SimpleRNN(64, activation='tanh', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Dense(32, activation='relu'),
    Dense(1)
])
rnn_model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])

# LSTM example
lstm_model = Sequential([
    LSTM(64, activation='tanh', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Dense(32, activation='relu'),
    Dense(1)
])
lstm_model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
history = rnn_model.fit(X_train_rnn, y_train,
                        validation_data=(X_test_rnn, y_test),
                        epochs=200, batch_size=32, callbacks=[early_stop])


Epoch 1/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 675.5500 - mae: 25.7985 - val_loss: 630.8401 - val_mae: 24.9399
Epoch 2/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 635.5869 - mae: 25.0124 - val_loss: 594.1142 - val_mae: 24.1898
Epoch 3/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 598.7742 - mae: 24.2616 - val_loss: 550.7097 - val_mae: 23.2701
Epoch 4/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 552.5092 - mae: 23.2803 - val_loss: 493.7808 - val_mae: 21.9996
Epoch 5/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 491.9816 - mae: 21.9229 - val_loss: 420.9966 - val_mae: 20.2500
Epoch 6/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 415.5284 - mae: 20.0659 - val_loss: 333.4238 - val_mae: 17.9069
Epoch 7/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 325.4215 - mae: 17.6097 - val_loss: 237.8014 - val_mae: 14.9227
Epoch 8/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 229.5388 - mae: 14.5290 - val_loss: 147.1060 - val_mae: 11.4036
Epoch 9/200
24/

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

y_pred = rnn_model.predict(X_test_rnn)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print(f"R² Score: {r2}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
R² Score: 0.39223327364900895
MSE: 5.4387086787119
RMSE: 2.332103916790995
MAE: 1.8533093983762694


In [ ]:
import tensorflow as tf, numpy as np, random, os
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


In [ ]:
import tensorflow as tf, numpy as np, random, os
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model.fit(X_train_rnn, y_train, validation_data=(X_test_rnn, y_test), epochs=200, callbacks=[es])


NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM

# Example RNN Model
model = Sequential()
model.add(SimpleRNN(64, activation='tanh', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # regression output

model.compile(optimizer='adam', loss='mse', metrics=['mae'])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
    X_train_rnn, y_train,
    validation_data=(X_test_rnn, y_test),
    epochs=200,
    callbacks=[es],
    batch_size=32
)


Epoch 1/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 675.5500 - mae: 25.7985 - val_loss: 630.8401 - val_mae: 24.9399
Epoch 2/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 635.5869 - mae: 25.0124 - val_loss: 594.1142 - val_mae: 24.1898
Epoch 3/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 598.7742 - mae: 24.2616 - val_loss: 550.7097 - val_mae: 23.2701
Epoch 4/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 552.5092 - mae: 23.2803 - val_loss: 493.7808 - val_mae: 21.9996
Epoch 5/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 491.9816 - mae: 21.9229 - val_loss: 420.9966 - val_mae: 20.2500
Epoch 6/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 415.5284 - mae: 20.0659 - val_loss: 333.4238 - val_mae: 17.9069
Epoch 7/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 325.4215 - mae: 17.6097 - val_loss: 237.8014 - val_mae: 14.9227
Epoch 8/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 229.5388 - mae: 14.5290 - val_loss: 147.1060 - val_mae: 11.4036
Epoch 9/200
24/

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

y_pred = model.predict(X_test_rnn)

print("R²:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
R²: 0.39223327364900895
MSE: 5.4387086787119
MAE: 1.8533093983762694
